In [27]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from cuml.ensemble import RandomForestRegressor as cuRF
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import time

In [28]:
# Загрузка данных
data = pd.read_csv('data/all_youtube_analytics.csv')  # Замените на путь к вашему файлу

# Предобработка данных
# Кодируем категориальные данные и заполняем пропущенные значения
data = pd.get_dummies(data, drop_first=True)
data = data.fillna(data.mean())

# Разделение данных на признаки и целевую переменную
X = data.drop('likes', axis=1)  # Замените 'likes' на название вашей целевой переменной
y = data['likes']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Приведение данных к типу float32 для работы с CuML
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')

In [29]:
# Реализация Random Forest
print("Training Random Forest")
start_time = time.time()
rf_model = cuRF(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_time = time.time() - start_time

# Предсказания и оценка модели
rf_predictions = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print(f"Random Forest MSE: {rf_mse:.4f}, R2: {rf_r2:.4f}, Time: {rf_time:.2f} seconds")

Training Random Forest...


KeyboardInterrupt: 

In [ ]:
# Реализация XGBoost
print("Training XGBoost")
start_time = time.time()
xgb_model = XGBRegressor(n_estimators=100, random_state=42, tree_method='gpu_hist')  # Используем GPU
xgb_model.fit(X_train, y_train)
xgb_time = time.time() - start_time

# Предсказания и оценка модели
xgb_predictions = xgb_model.predict(X_test)
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_r2 = r2_score(y_test, xgb_predictions)

print(f"XGBoost MSE: {xgb_mse:.4f}, R2: {xgb_r2:.4f}, Time: {xgb_time:.2f} seconds")

In [ ]:
# Сравнение результатов
results = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost'],
    'MSE': [rf_mse, xgb_mse],
    'R2': [rf_r2, xgb_r2],
    'Time (seconds)': [rf_time, xgb_time]
})

print("\nComparison of Models:")
print(results)